# Step 4 - Analysis of bicycle network results
## Project: Growing Urban Bicycle Networks with LTNs

This notebook takes the existing infrastructure, the results from 03_poi_based_generation and calculates/analyzes a number of measures:
* cost (length)
* coverage  
* directness  
* efficiency
* overlap with existing networks

Contact: Chris Larkin (c.larkin@ncl.ac.uk)  
Created: 2020-07-08  
Last modified: 2024-09-24

## Preliminaries

### Parameters

In [ ]:
debug = True # If True, will produce plots and/or verbose output to double-check
rerun_existing = True # If True, will re-run the costly analysis of existing infra even if files already exist.
%run -i "../parameters/parameters.py"

: 

### Setup

In [ ]:
%run -i path.py
%run -i setup.py
if not debug: # Only do this if sure the code is bug-free!
    warnings.filterwarnings('ignore')

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Analyze existing infrastructure

In [19]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"): 
    # output_place is one static file for the existing city. This can be compared to the generated infrastructure.
    # Make a check if this file was already generated - it only needs to be done once. If not, generate it:
    if weighting == True:
        filename = placeid + "_existing_weighted.csv"
    else:
        filename = placeid + "_existing.csv"
    if rerun_existing or not os.path.isfile(PATH["results"] + placeid + "/" + filename):
        print(placeid + ": Analyzing existing infrastructure... (this can take several minutes)")
        empty_metrics = {
                         "length":0,
                         "length_lcc":0,
                         "coverage": 0,
                         "directness": 0,
                         "directness_lcc": 0,
                         "poi_coverage": 0,
                         "components": 0,
                         "efficiency_global": 0,
                         "efficiency_local": 0,
                         "efficiency_global_routed": 0,
                         "efficiency_local_routed": 0,
                         "directness_lcc_linkwise": 0,
                         "directness_all_linkwise": 0
                        }
        output_place = {}
        for networktype in networktypes:
            output_place[networktype] = copy.deepcopy(empty_metrics)

        # Analyze all networks     
        Gs = {}
        for networktype in networktypes:
            try:
                if networktype != "biketrack_onstreet" and networktype != "bikeable_offstreet":
                    Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
                    Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified")
                elif networktype == "biketrack_onstreet":
                    Gs[networktype] = intersect_igraphs(Gs["biketrack"], Gs["carall"])
                    Gs[networktype + "_simplified"] = intersect_igraphs(Gs["biketrack_simplified"], Gs["carall_simplified"])
                elif networktype == "bikeable_offstreet":
                    G_temp = copy.deepcopy(Gs["bikeable"])
                    delete_overlaps(G_temp, Gs["carall"])
                    Gs[networktype] = G_temp
                    G_temp = copy.deepcopy(Gs["bikeable_simplified"])
                    delete_overlaps(G_temp, Gs["carall_simplified"])
                    Gs[networktype + "_simplified"] = G_temp
                elif networktype == "ltnstreets":
                    Gs[networktype] = intersect_igraphs(Gs["ltnstreets"], Gs["carall"])
                    Gs[networktype + "_simplified"] = intersect_igraphs(Gs["ltnstreets_simplified"], Gs["carall_simplified"])
            except:
                pass
        
        with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
            nnids = [int(line.rstrip()) for line in f]

            
        covs = {}
        for networktype in tqdm(networktypes, desc = "Networks", leave = False):
            if debug: print(placeid + ": Analyzing results: " + networktype)
            #try:
            metrics, cov = calculate_metrics(Gs[networktype], Gs[networktype + "_simplified"], Gs['carall'], nnids, empty_metrics, buffer_walk, numnodepairs, debug)
            for key, val in metrics.items():
                output_place[networktype][key] = val
            covs[networktype] = cov
            #except:
            #  print(networktype + " is empty")

        # Save the covers
        suffix = "existing_covers"
        write_result_covers(covs, "pickle", placeid, suffix, weighting=weighting)

        # Write to CSV
        suffix = "existing"
        write_result_covers(output_place, "dictnested", placeid, suffix, dictnested=empty_metrics, weighting=weighting)

    else:
        print(placeid + ": Existing infrastructure has already been analyzed")

Cities:   0%|          | 0/1 [00:00<?, ?it/s]

newcastle: Analyzing existing infrastructure... (this can take several minutes)


Networks:   0%|          | 0/6 [00:00<?, ?it/s]

Problem with efficiency_global_routed.
Problem with efficiency_local_routed.
Problem with efficiency_global_routed.
Problem with efficiency_local_routed.
Problem with efficiency_global_routed.
Problem with efficiency_local_routed.
Problem with efficiency_global_routed.
Problem with efficiency_local_routed.
Problem with efficiency_global_routed.
Problem with efficiency_local_routed.
Problem with efficiency_global_routed.
Problem with efficiency_local_routed.


## Analyze POI based results

In [20]:
# # debug testing
# for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
#     print(placeid + ": Analyzing results")

#     # Load networks
#     G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall', weighting=weighting)
#     # Assuming G_carall is your igraph object
#     for i in range(min(5, len(G_carall.es))):  # Ensure you only print up to 5 edges if there are fewer
#         edge = G_carall.es[i]
#         print(f"Edge {i}:")
#         print(edge.attributes())  # Prints all attributes of the edge
#        # Restore orignal edge lengths
#     if weighting == True:
#         restore_original_lengths(G_carall)
#     # Assuming G_carall is your igraph object
#     for i in range(min(5, len(G_carall.es))):  # Ensure you only print up to 5 edges if there are fewer
#         edge = G_carall.es[i]
#         print(f"Edge {i}:")
#         print(edge.attributes())  # Prints all attributes of the edge

        


In [21]:
# for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
#     # load neighbourhood streets
#     neighbourhoods = load_neighbourhoods(PATH["data"] + placeid + "/")
#     # Initialize lists to store the results
#     all_nodes, all_edges = [], []

#     # Loop through the neighbourhoods dictionary
#     for name, gdf in neighbourhoods.items():
#         if gdf.empty:
#             print(f"Warning: The GeoDataFrame for {name} is empty. Skipping...")
#             continue
        
#         print(f"Processing neighbourhoods in: {name}")

#         # Run the function on each polygon and collect results
#         for _, polygon in gdf.iterrows():
#             nodes, edges = get_neighbourhood_streets(polygon.geometry, debug)
#             all_nodes.append(nodes)
#             all_edges.append(edges)

#     # Combine the results into single GeoDataFrames
#     combined_nodes = pd.concat(all_nodes, ignore_index=True)
#     combined_edges = pd.concat(all_edges, ignore_index=True)


In [82]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": Analyzing results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall', weighting=weighting)
    try:
        if weighting == True:
            restore_original_lengths(G_carall) # unweight the graph for calcluations of length etc...
    except:
        pass

    # load neighbourhood streets
    G_neighbourhoods = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'ltnstreets')

    Gexisting = {}
    for networktype in ["biketrack", "bikeable"]:
        try:
            Gexisting[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
        except:
            print(networktype + " is empty")
        
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
            
    # Load results
    # Modify filename based on weighting flag
    weighting_str = "_weighted" if weighting else ""

    # Load results with the appropriate filename considering the weighting flag
    filename_base = f"{placeid}_poi_{poi_source}_{prune_measure}"
    # Load results with the appropriate filename considering the weighting flag
    filename = filename_base + f"{'_weighted' if weighting else ''}.pickle"
    resultfile = open(PATH["results"] + placeid + "/" + filename, 'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    
    if debug: 
        pp.pprint(res)
    
        
     
    # Calculate
    # output contains lists for all the prune_quantile values of the corresponding results
    output, covs = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting, G_neighbourhoods)
    #output_MST, cov_MST = calculate_metrics(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, G_neighbourhoods)
        
    # Save the covers
    write_result(covs, "pickle", placeid, poi_source, prune_measure, "_covers.pickle", weighting=weighting)
    #write_result(cov_MST, "pickle", placeid, poi_source, prune_measure, "_cover_mst.pickle", weighting=weighting)
        
    # Write to CSV
    write_result(output, "dict", placeid, poi_source, prune_measure, ".csv", weighting=weighting)
    #write_result(output_MST, "dict", placeid, poi_source, "", "_mst.csv", weighting=weighting)


    
#     write_result(output_carminusbike, "dict", placeid, poi_source, prune_measure, "_carminusbike.csv")
#     write_result(output_carconstrictedbike, "dict", placeid, poi_source, prune_measure, "_carconstrictedbike.csv")


Cities:   0%|          | 0/1 [00:00<?, ?it/s]

newcastle: Analyzing results
{   'GT_abstracts': [   <igraph.Graph object at 0x00000199D01D9050>,
    'GTs': [   <igraph.Graph object at 0x00000199A0EE6850>,
    'placeid': 'newcastle',
    'poi_source': 'LTNs_tessellation',
    'prune_measure': 'betweenness',
    'prune_quantiles': [   30000,
                           35000,
                           40000,
                           45000,
                           50000,
                           55000,
                           60000,
                           65000,
                           70000,
                           75000,
                           80000,
                           85000,
                           90000,
                           95000,
                           100000,
                           105000,
                           110000,
                           115000,
                           120000,
                           125000,
                           130000,
                  

Bicycle networks:   0%|          | 0/114 [00:00<?, ?it/s]

Calculating bike network metrics for quantile 30000
IGRAPH D--- 1579 2766 -- 
+ attr: crs (g), _nx_name (v), x (v), y (v), _nx_multiedge_key (e), length (e), osmid (e)
IGRAPH D-W- 19 16 -- 
+ attr: _nx_name (v), x (v), x_original (v), y (v), y_original (v), _nx_multiedge_key (e), betweeness (e), geometry (e), weight (e)
IGRAPH U-W- 36848 39014 -- 
+ attr: id (v), x (v), y (v), ori_length (e), osmid (e), weight (e)
Calculating efficiency...
EG:  0.01272225025293783
EG_id 0.11358446685163234
EG:  0.0014116075625932127
EG_id 0.009061316702298141
EG:  0.0017023511321974676
EG_id 0.00906131670229814
EG:  0.0
EG_id 0.0014744825898279
EG:  0.0
EG_id 0.0011971238381543203
EG:  0.0
EG_id 0
EG_id is zero. Returning default efficiency value.
EG:  0.0
EG_id 0.0014244399803370485
EG:  0.0
EG_id 0.0004331732636556263
EG:  0.0020800679893380647
EG_id 0.003570588531333087
EG:  0.0014116075625932127
EG_id 0.0049248601550865075
Calculating efficiency (routed)...
Problem with efficiency_global_routed.
Pr

In [ ]:
## debug testing

In [80]:
def calculate_efficiency_global(G, numnodepairs = 500, normalized = True):
    """Calculates global network efficiency.
    If there are more than numnodepairs nodes, measure over pairings of a 
    random sample of numnodepairs nodes.
    """
    if "x" not in G.vs.attributes() or "y" not in G.vs.attributes():
        raise KeyError("Graph vertices are missing 'x' or 'y' attributes.")


    if G is None: return 0
    if G.vcount() > numnodepairs:
        nodeindices = random.sample(list(G.vs.indices), numnodepairs)
    else:
        nodeindices = list(G.vs.indices)
    d_ij = G.shortest_paths(source = nodeindices, target = nodeindices, weights = "weight")
    d_ij = [item for sublist in d_ij for item in sublist] # flatten

    ### Check if d_ij contains valid distances
    if not d_ij: return 0  # No distances available
    ###

    

    EG = sum([1/d for d in d_ij if d != 0])

    if debug:
        print("EG: ", EG)
    if not normalized: return EG
    pairs = list(itertools.permutations(nodeindices, 2))
    if len(pairs) < 1: return 0
    l_ij = dist_vector([(G.vs[p[0]]["y"], G.vs[p[0]]["x"]) for p in pairs],
                            [(G.vs[p[1]]["y"], G.vs[p[1]]["x"]) for p in pairs]) # must be in format lat,lon = y,x
    EG_id = sum([1/l for l in l_ij if l != 0])

    if debug:
        print("EG_id", EG_id)
    # re comment this block later
    #if (EG / EG_id) > 1: # This should not be allowed to happen!
    #    pp.pprint(d_ij)
    #    pp.pprint(l_ij)
    #    pp.pprint([e for e in G.es])
    #    print(pairs)
    #    print([(G.vs[p[0]]["y"], G.vs[p[0]]["x"]) for p in pairs],
    #                         [(G.vs[p[1]]["y"], G.vs[p[1]]["x"]) for p in pairs]) # must be in format lat,lon = y,x
    #    print(EG, EG_id)
    #   sys.exit()
    # assert EG / EG_id <= 1, "Normalized EG > 1. This should not be possible."


    if EG_id == 0:
        print("EG_id is zero. Returning default efficiency value.")
        return 0  # Or another appropriate default value
    return EG / EG_id


def calculate_efficiency_local(G, numnodepairs = 500, normalized = True):
    """Calculates local network efficiency.
    If there are more than numnodepairs nodes, measure over pairings of a 
    random sample of numnodepairs nodes.
    """

    if G is None: return 0
    if G.vcount() > numnodepairs:
        nodeindices = random.sample(list(G.vs.indices), numnodepairs)
    else:
        nodeindices = list(G.vs.indices)
    EGi = []
    vcounts = []
    ecounts = []
    for i in nodeindices:
        if len(G.neighbors(i)) > 1: # If we have a nontrivial neighborhood
            G_induced = G.induced_subgraph(G.neighbors(i))
            EGi.append(calculate_efficiency_global(G_induced, numnodepairs, normalized))
    return listmean(EGi)


def calculate_metrics(
    G, GT_abstract, G_big, nnids, calcmetrics={"length": 0, "length_lcc": 0, "coverage": 0, "directness": 0,
                                               "directness_lcc": 0, "poi_coverage": 0, "components": 0,
                                               "overlap_biketrack": 0, "overlap_bikeable": 0, "efficiency_global": 0,
                                               "efficiency_local": 0, "directness_lcc_linkwise": 0,
                                               "directness_all_linkwise": 0, "overlap_neighbourhood": 0},
    buffer_walk=500, numnodepairs=500, verbose=False, return_cov=True, G_prev=ig.Graph(),
    cov_prev=Polygon(), ignore_GT_abstract=False, Gexisting={}, Gneighbourhoods=None):
    """Calculates all metrics (using the keys from calcmetrics)."""

    output = {key: 0 for key in calcmetrics}
    cov = Polygon()

    print(GT.summary())
    print(GT_abstract.summary())
    print(G_big.summary())



    # Check that the graph has links (sometimes we have an isolated node)
    if G.ecount() > 0 and GT_abstract.ecount() > 0:
        # Get LCC
        cl = G.clusters()
        LCC = cl.giant()

        # EFFICIENCY 
        if not ignore_GT_abstract:
            if verbose and ("efficiency_global" in calcmetrics or "efficiency_local" in calcmetrics): print("Calculating efficiency...")
            if "efficiency_global" in calcmetrics:
                output["efficiency_global"] = calculate_efficiency_global(GT_abstract, numnodepairs)
            if "efficiency_local" in calcmetrics:
                output["efficiency_local"] = calculate_efficiency_local(GT_abstract, numnodepairs) 
        
        # EFFICIENCY ROUTED
        if verbose and ("efficiency_global_routed" in calcmetrics or "efficiency_local_routed" in calcmetrics): print("Calculating efficiency (routed)...")
        if "efficiency_global_routed" in calcmetrics:
            try:
                output["efficiency_global_routed"] = calculate_efficiency_global(simplify_ig(G), numnodepairs)
            except:
                print("Problem with efficiency_global_routed.") 
        if "efficiency_local_routed" in calcmetrics:
            try:
                output["efficiency_local_routed"] = calculate_efficiency_local(simplify_ig(G), numnodepairs)
            except:
                print("Problem with efficiency_local_routed.")

        # LENGTH
        if verbose and ("length" in calcmetrics or "length_lcc" in calcmetrics): print("Calculating length...")
        if "length" in calcmetrics:
            output["length"] = sum([e['weight'] for e in G.es])
        if "length_lcc" in calcmetrics:
            if len(cl) > 1:
                output["length_lcc"] = sum([e['weight'] for e in LCC.es])
            else:
                output["length_lcc"] = output["length"]
        
        # COVERAGE
        if "coverage" in calcmetrics:
            if verbose: print("Calculating coverage...")
            covered_area, cov = calculate_coverage_edges(G, buffer_walk, return_cov, G_prev, cov_prev)
            output["coverage"] = covered_area

            # OVERLAP WITH EXISTING NETS
            if Gexisting:
                if "overlap_biketrack" in calcmetrics:
                    try:
                        output["overlap_biketrack"] = edge_lengths(intersect_igraphs(Gexisting["biketrack"], G))
                    except:  # If there is not bike infrastructure, set to zero
                        output["overlap_biketrack"] = 0
                if "overlap_bikeable" in calcmetrics:
                    try:
                        output["overlap_bikeable"] = edge_lengths(intersect_igraphs(Gexisting["bikeable"], G))
                    except:  # If there is not bikeable infrastructure, set to zero
                        output["overlap_bikeable"] = 0

        # OVERLAP WITH NEIGHBOURHOOD NETWORK
        if Gneighbourhoods and "overlap_neighbourhood" in calcmetrics:
            if verbose: print("Calculating overlap_neighbourhood...")
            try:
                output["overlap_neighbourhood"] = edge_lengths(intersect_igraphs(Gneighbourhoods, G))
            except:  # If there are issues with intersecting graphs, set to zero
                output["overlap_neighbourhood"] = 0

        # POI COVERAGE
        if "poi_coverage" in calcmetrics:
            if verbose: print("Calculating POI coverage...")
            output["poi_coverage"] = calculate_poiscovered(G_big, cov, nnids)

        # COMPONENTS
        if "components" in calcmetrics:
            if verbose: print("Calculating components...")
            output["components"] = len(list(G.components()))
        
        # DIRECTNESS
        if verbose and ("directness" in calcmetrics or "directness_lcc" in calcmetrics): print("Calculating directness...")
        if "directness" in calcmetrics:
            output["directness"] = calculate_directness(G, numnodepairs)
        if "directness_lcc" in calcmetrics:
            if len(cl) > 1:
                output["directness_lcc"] = calculate_directness(LCC, numnodepairs)
            else:
                output["directness_lcc"] = output["directness"]

        # DIRECTNESS LINKWISE
        if verbose and ("directness_lcc_linkwise" in calcmetrics): print("Calculating directness linkwise...")
        if "directness_lcc_linkwise" in calcmetrics:
            if len(cl) > 1:
                output["directness_lcc_linkwise"] = calculate_directness_linkwise(LCC, numnodepairs)
            else:
                output["directness_lcc_linkwise"] = calculate_directness_linkwise(G, numnodepairs)
        if verbose and ("directness_all_linkwise" in calcmetrics): print("Calculating directness linkwise (all components)...")
        if "directness_all_linkwise" in calcmetrics:
            if "directness_lcc_linkwise" in calcmetrics and len(cl) <= 1:
                output["directness_all_linkwise"] = output["directness_lcc_linkwise"]
            else:  # we have >1 components
                output["directness_all_linkwise"] = calculate_directness_linkwise(G, numnodepairs)

    if return_cov: 
        return output, cov
    else:
        return output



def calculate_metrics_additively(
    Gs, GT_abstracts, prune_quantiles, G_big, nnids, buffer_walk=500, numnodepairs=500, verbose=False, 
    return_cov=True, Gexisting={}, Gneighbourhoods=None,
    output={
        "length": [], "length_lcc": [], "coverage": [], "directness": [], "directness_lcc": [],
        "poi_coverage": [], "components": [], "overlap_biketrack": [], "overlap_bikeable": [],
        "efficiency_global": [], "efficiency_local": [], "efficiency_global_routed": [], 
        "efficiency_local_routed": [], "directness_lcc_linkwise": [], "directness_all_linkwise": [],
        "overlap_neighbourhood": []  # Add the new metric here
    }
):
    """Calculates all metrics, additively. 
    Coverage differences are calculated in every step instead of the whole coverage.
    """

    # BICYCLE NETWORKS
    covs = {}  # Covers using buffer_walk
    cov_prev = Polygon()
    GT_prev = ig.Graph()

    for GT, GT_abstract, prune_quantile in zip(Gs, GT_abstracts, tqdm(prune_quantiles, desc="Bicycle networks", leave=False)):
        if verbose: print("Calculating bike network metrics for quantile " + str(prune_quantile))
        metrics, cov = calculate_metrics(
            GT, GT_abstract, G_big, nnids, output, buffer_walk, numnodepairs, verbose, 
            return_cov, GT_prev, cov_prev, False, Gexisting, Gneighbourhoods
        )

        for key in output.keys():
            output[key].append(metrics[key])
        covs[prune_quantile] = cov
        cov_prev = copy.deepcopy(cov)
        GT_prev = copy.deepcopy(GT)

    return output, covs

In [41]:
print(f"Graph {networktype} summary:", Gexisting[networktype].summary())
print(f"Edges: {Gexisting[networktype].ecount()}, Nodes: {Gexisting[networktype].vcount()}")


Graph bikeable summary: IGRAPH U-W- 15767 15831 -- 
+ attr: id (v), x (v), y (v), osmid (e), weight (e)
Edges: 15831, Nodes: 15767


In [ ]:
## end of debug testing

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": Analyzing results")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall', weighting=weighting)
    try:
        if weighting == True:
            restore_original_lengths(G_carall) # unweight the graph for calcluations of length etc...
    except:
        pass

    # load neighbourhood streets
    G_neighbourhoods = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'ltnstreets')

    Gexisting = {}
    for networktype in ["biketrack", "bikeable"]:
        try:
            Gexisting[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)
        except:
            print(networktype + " is empty")
        
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
            
    # Load results
    # Modify filename based on weighting flag
    weighting_str = "_weighted" if weighting else ""

    # Load results with the appropriate filename considering the weighting flag
    filename_base = f"{placeid}_poi_{poi_source}_{prune_measure}"
    # Load results with the appropriate filename considering the weighting flag
    filename = filename_base + f"{'_weighted' if weighting else ''}.pickle"
    resultfile = open(PATH["results"] + placeid + "/" + filename, 'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    
    if debug: 
        pp.pprint(res)
         
    # Calculate
    # output contains lists for all the prune_quantile values of the corresponding results
    output, covs = calculate_metrics_additively(res["GTs"], res["GT_abstracts"], res["prune_quantiles"], G_carall, nnids, buffer_walk, numnodepairs, debug, True, Gexisting, G_neighbourhoods)
    output_MST, cov_MST = calculate_metrics(res["MST"], res["MST_abstract"], G_carall, nnids, output, buffer_walk, numnodepairs, debug, True, ig.Graph(), Polygon(), False, G_neighbourhoods)
        
    # Save the covers
    write_result(covs, "pickle", placeid, poi_source, prune_measure, "_covers.pickle", weighting=weighting)
    write_result(cov_MST, "pickle", placeid, poi_source, prune_measure, "_cover_mst.pickle", weighting=weighting)
        
    # Write to CSV
    write_result(output, "dict", placeid, poi_source, prune_measure, ".csv", weighting=weighting)
    write_result(output_MST, "dict", placeid, poi_source, "", "_mst.csv", weighting=weighting)


    
#     write_result(output_carminusbike, "dict", placeid, poi_source, prune_measure, "_carminusbike.csv")
#     write_result(output_carconstrictedbike, "dict", placeid, poi_source, prune_measure, "_carconstrictedbike.csv")


Cities:   0%|          | 0/1 [00:00<?, ?it/s]

newcastle: Analyzing results
{   'GT_abstracts': [   <igraph.Graph object at 0x00000199A00C1350>,
    'GTs': [   <igraph.Graph object at 0x00000199A0EE6750>,
    'placeid': 'newcastle',
    'poi_source': 'LTNs_tessellation',
    'prune_measure': 'betweenness',
    'prune_quantiles': [   30000,
                           35000,
                           40000,
                           45000,
                           50000,
                           55000,
                           60000,
                           65000,
                           70000,
                           75000,
                           80000,
                           85000,
                           90000,
                           95000,
                           100000,
                           105000,
                           110000,
                           115000,
                           120000,
                           125000,
                           130000,
                  

Bicycle networks:   0%|          | 0/114 [00:00<?, ?it/s]

Calculating bike network metrics for quantile 30000
IGRAPH D--- 1579 2766 -- 
+ attr: crs (g), _nx_name (v), x (v), y (v), _nx_multiedge_key (e), length (e), osmid (e)
IGRAPH D-W- 19 16 -- 
+ attr: _nx_name (v), x (v), x_original (v), y (v), y_original (v), _nx_multiedge_key (e), betweeness (e), geometry (e), weight (e)
IGRAPH U-W- 36848 39014 -- 
+ attr: id (v), x (v), y (v), ori_length (e), osmid (e), weight (e)
Calculating efficiency...
EG:  0.01272225025293783
EG_id 0.11358446685163234
EG:  0.0014116075625932127
EG_id 0.009061316702298141
EG:  0.0017023511321974676
EG_id 0.00906131670229814
EG:  0.0
EG_id 0.0014744825898279
EG:  0.0
EG_id 0.0011971238381543203
EG:  0.0
EG_id 0
EG_id is zero. Returning default efficiency value.
EG:  0.0
EG_id 0.0014244399803370485
EG:  0.0
EG_id 0.0004331732636556263
EG:  0.0020800679893380647
EG_id 0.003570588531333087
EG:  0.0014116075625932127
EG_id 0.0049248601550865075
Calculating efficiency (routed)...
Problem with efficiency_global_routed.
Pr

KeyError: 'MST'

: 

In [ ]:
Audio(sound_file, autoplay=True) 